<a href="https://colab.research.google.com/github/faizanali02/googlecolab/blob/main/Assingnment07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ===============================================
# LAB 05: Optimization Algorithms
# Activities 1–3 + Truck Loading Exercise
# ===============================================

import random, math, itertools

# ==========================================================
# ACTIVITY 1: HILL CLIMBING (1D Optimization)
# ==========================================================

def objective(x):
    """Objective function to maximize"""
    return -(x - 3) ** 2 + 9  # Maximum at x = 3

def hill_climbing():
    current_x = random.uniform(-10, 10)
    current_value = objective(current_x)
    print("Initial x:", round(current_x, 3), "Value:", round(current_value, 3))

    for i in range(100):
        neighbor = current_x + random.uniform(-1, 1)
        neighbor_value = objective(neighbor)
        if neighbor_value > current_value:
            current_x, current_value = neighbor, neighbor_value

    print("\n=== Hill Climbing Result ===")
    print("Best x =", round(current_x, 3))
    print("Maximum Value =", round(current_value, 3))
    return current_x, current_value

# ==========================================================
# ACTIVITY 2: SIMULATED ANNEALING
# ==========================================================

def simulated_annealing():
    current_x = random.uniform(-10, 10)
    current_value = objective(current_x)
    T = 100
    cooling_rate = 0.95
    print("Initial x:", round(current_x, 3), "Value:", round(current_value, 3))

    while T > 0.001:
        neighbor = current_x + random.uniform(-1, 1)
        neighbor_value = objective(neighbor)
        delta = neighbor_value - current_value

        # Accept move if better OR with some probability
        if delta > 0 or random.random() < math.exp(delta / T):
            current_x, current_value = neighbor, neighbor_value

        T *= cooling_rate

    print("\n=== Simulated Annealing Result ===")
    print("Best x =", round(current_x, 3))
    print("Maximum Value =", round(current_value, 3))
    return current_x, current_value

# ==========================================================
# ACTIVITY 3: GENETIC ALGORITHM
# ==========================================================

def fitness(x):
    return -(x - 3) ** 2 + 9  # maximize

def genetic_algorithm():
    population = [random.uniform(-10, 10) for _ in range(10)]
    for generation in range(50):
        population = sorted(population, key=fitness, reverse=True)
        next_gen = population[:2]  # elitism
        while len(next_gen) < 10:
            p1, p2 = random.sample(population[:5], 2)
            child = (p1 + p2) / 2  # crossover
            child += random.uniform(-0.5, 0.5)  # mutation
            next_gen.append(child)
        population = next_gen
    best = max(population, key=fitness)
    print("\n=== Genetic Algorithm Result ===")
    print("Best x =", round(best, 3))
    print("Maximum Value =", round(fitness(best), 3))
    return best, fitness(best)

# ==========================================================
# EXERCISE: TRUCK LOADING OPTIMIZATION (0/1 Knapsack)
# ==========================================================

products = [
    ("Refrigerator A", 0.751, 1000),
    ("Cell Phone", 0.001, 2000),
    ("TV", 0.400, 1500),
    ("Fan", 0.500, 300),
    ("Microwave", 0.350, 800),
    ("Refrigerator B", 0.600, 900),
    ("Notebook", 0.200, 1200),
    ("Speaker", 0.150, 700),
    ("Washing Machine", 0.800, 1100),
    ("Blender", 0.200, 400),
    ("Printer", 0.250, 500),
    ("Camera", 0.050, 1400),
    ("Air Purifier", 0.300, 850),
    ("Monitor", 0.218, 950),
]

capacity = 3.0

def evaluate(solution):
    """Compute total value and volume for a given binary solution"""
    total_value, total_vol = 0, 0
    for bit, (name, vol, val) in zip(solution, products):
        if bit:
            total_value += val
            total_vol += vol
    if total_vol > capacity:
        return 0, total_vol  # infeasible
    return total_value, total_vol

# -------------------------------
# Brute-force optimal for comparison
# -------------------------------
def brute_force_optimal():
    best_val, best_sol, best_vol = 0, None, 0
    for combo in itertools.product([0, 1], repeat=len(products)):
        val, vol = evaluate(combo)
        if val > best_val and vol <= capacity:
            best_val, best_sol, best_vol = val, combo, vol
    return best_val, best_sol, best_vol

# -------------------------------
# Hill Climbing for Truck Loading
# -------------------------------
def hill_climb_knapsack():
    current = [random.choice([0, 1]) for _ in products]
    current_val, current_vol = evaluate(current)
    for _ in range(500):
        neighbor = current[:]
        i = random.randrange(len(products))
        neighbor[i] = 1 - neighbor[i]
        val, vol = evaluate(neighbor)
        if val > current_val:
            current, current_val, current_vol = neighbor, val, vol
    return current_val, current, current_vol

# -------------------------------
# Simulated Annealing for Truck Loading
# -------------------------------
def simulated_annealing_knapsack():
    current = [random.choice([0, 1]) for _ in products]
    current_val, current_vol = evaluate(current)
    T, cooling_rate = 100, 0.95
    while T > 0.001:
        neighbor = current[:]
        i = random.randrange(len(products))
        neighbor[i] = 1 - neighbor[i]
        val, vol = evaluate(neighbor)
        delta = val - current_val
        if delta > 0 or random.random() < math.exp(delta / T):
            current, current_val, current_vol = neighbor, val, vol
        T *= cooling_rate
    return current_val, current, current_vol

# -------------------------------
# Genetic Algorithm for Truck Loading
# -------------------------------
def genetic_algorithm_knapsack():
    population = [[random.choice([0, 1]) for _ in products] for _ in range(20)]

    def mutate(sol):
        sol = sol[:]
        i = random.randrange(len(sol))
        sol[i] = 1 - sol[i]
        return sol

    for generation in range(100):
        population = sorted(population, key=lambda s: evaluate(s)[0], reverse=True)
        next_gen = population[:4]  # elitism
        while len(next_gen) < 20:
            p1, p2 = random.sample(population[:10], 2)
            cut = random.randint(1, len(products) - 1)
            child = p1[:cut] + p2[cut:]
            if random.random() < 0.3:
                child = mutate(child)
            next_gen.append(child)
        population = next_gen
    best = max(population, key=lambda s: evaluate(s)[0])
    val, vol = evaluate(best)
    return val, best, vol

# ==========================================================
# MAIN EXECUTION
# ==========================================================

print("=== ACTIVITY 1–3 RESULTS ===")
hill_climbing()
simulated_annealing()
genetic_algorithm()

print("\n=== TRUCK LOADING EXERCISE ===")
opt_val, opt_sol, opt_vol = brute_force_optimal()
print("\nBrute-force Optimal:")
print("Value =", opt_val, "Volume =", round(opt_vol, 3))

hc_val, hc_sol, hc_vol = hill_climb_knapsack()
print("\nHill Climbing:")
print("Value =", hc_val, "Volume =", round(hc_vol, 3))

sa_val, sa_sol, sa_vol = simulated_annealing_knapsack()
print("\nSimulated Annealing:")
print("Value =", sa_val, "Volume =", round(sa_vol, 3))

ga_val, ga_sol, ga_vol = genetic_algorithm_knapsack()
print("\nGenetic Algorithm:")
print("Value =", ga_val, "Volume =", round(ga_vol, 3))

print("\n=== SELECTED PRODUCTS (GENETIC ALGORITHM) ===")
for bit, (name, vol, val) in zip(ga_sol, products):
    if bit:
        print(f"- {name} | Volume: {vol} | Value: {val}")

print("\nComparison Summary:")
print(f"Brute-force = {opt_val}, Hill Climb = {hc_val}, Simulated Annealing = {sa_val}, Genetic = {ga_val}")


=== ACTIVITY 1–3 RESULTS ===
Initial x: 8.121 Value: -17.223

=== Hill Climbing Result ===
Best x = 3.007
Maximum Value = 9.0
Initial x: 1.271 Value: 6.01

=== Simulated Annealing Result ===
Best x = 3.038
Maximum Value = 8.999

=== Genetic Algorithm Result ===
Best x = 2.997
Maximum Value = 9.0

=== TRUCK LOADING EXERCISE ===

Brute-force Optimal:
Value = 11400 Volume = 2.919

Hill Climbing:
Value = 11400 Volume = 2.919

Simulated Annealing:
Value = 5850 Volume = 3.0

Genetic Algorithm:
Value = 10950 Volume = 2.97

=== SELECTED PRODUCTS (GENETIC ALGORITHM) ===
- Refrigerator A | Volume: 0.751 | Value: 1000
- Cell Phone | Volume: 0.001 | Value: 2000
- TV | Volume: 0.4 | Value: 1500
- Microwave | Volume: 0.35 | Value: 800
- Refrigerator B | Volume: 0.6 | Value: 900
- Notebook | Volume: 0.2 | Value: 1200
- Speaker | Volume: 0.15 | Value: 700
- Printer | Volume: 0.25 | Value: 500
- Camera | Volume: 0.05 | Value: 1400
- Monitor | Volume: 0.218 | Value: 950

Comparison Summary:
Brute-force 